In [108]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import *
import findspark
findspark.init()

In [109]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Python Spark SQL basic example") \
    .config("spark.driver.maxResultSize", "1g") \
    .getOrCreate()

In [110]:
df=spark.read.csv('datasets/canadadataset.csv',header=True)
df.show()

+----------+---------+---------+-------------+----------+---------------+--------------+----------+-----------+
|visit_high|visit_low|visit_num|   event_list|hit_source|post_visit_high|post_visit_low|        ds|exclude_hit|
+----------+---------+---------+-------------+----------+---------------+--------------+----------+-----------+
|         0|        0|       35| 108,11,125,1|         2|              0|             0|01-07-2023|          0|
|         0|        0|       35| 108,11,125,2|         2|              0|             0|02-07-2023|          0|
|         0|        0|       35| 108,11,125,3|         2|              0|             0|03-07-2023|          0|
|         0|        0|       35| 108,11,125,4|         2|              0|             0|04-07-2023|          0|
|         0|        0|       35| 108,11,125,5|         2|              0|             0|05-07-2023|          0|
|         0|        0|       35| 108,11,125,6|         2|              0|             0|06-07-2023|     

In [158]:
df_new=df.withColumn("visit_hi_id_new",expr("CASE WHEN visit_high='0' or visit_high IS NULL THEN post_visit_high ELSE Visit_high END"))\
         .withColumn("visit_low_id_new",expr("CASE WHEN visit_low='0' or visit_low IS NULL THEN post_visit_low ELSE Visit_low END"))
df_new1=df_new.withColumn("session_cnt", expr("CASE WHEN hit_source NOT IN ('5','7','8','9') then concat(visit_hi_id_new,'-',visit_low_id_new,'-',visit_num) ELSE NULL END"))
df2=df_new1.where((df_new1.ds).between('01-07-2023','31-07-2023'))
df3=df2.withColumn("ATC_RATE",split(col("event_list"),',').getItem(1))
        
#df22=df_new1.withColumn("ATC_RATE",col("event_list")==11).filter(col("ATC_RATE")==True)
#df22=df_new1.withColumn("ATC_RATE",df_new1.ds.between('01-07-2023','31-07-2023')&(df_new1.exclude_hit=='1')&(df_new1.event_list=='12'))  
#df3=df22.select(countDistinct(col("session_cnt")).alias("distn_cnt"))
df4=df3.agg(countDistinct(col("session_cnt")).alias("dst_cnt"))
df4.show(truncate=False)

+-------+
|dst_cnt|
+-------+
|3      |
+-------+



In [157]:
df3=df2.withColumn("ATC_RATE",split(col("event_list"),',').getItem(0)).select("ATC_RATE")
       
df3.show()

+--------+
|ATC_RATE|
+--------+
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
+--------+
only showing top 20 rows



In [112]:
# df_new=df.withColumn("visit_hi_id_new",expr("CASE WHEN visit_high='0' or visit_high IS NULL THEN post_visit_high ELSE Visit_high END"))\
#          .withColumn("visit_low_id_new",expr("CASE WHEN visit_low='0' or visit_low IS NULL THEN post_visit_low ELSE Visit_low END"))
# df_new1=df_new.withColumn("session_cnt", expr("CASE WHEN hit_source NOT IN ('5','7','8','9') then concat(visit_hi_id_new,'-',visit_low_id_new,'-',visit_num) ELSE NULL END"))\
# #df_new1.drop_duplicates(df_new1.session_cnt).show()
# #df_new1.show()

# df3=df_new1.select(countDistinct("session_cnt").alias("distn_cnt"),)
# df3.show(truncate=False)

In [113]:
df2=df_new1.where((df_new1.ds).between('01-07-2023','31-07-2023')&(df_new1.exclude_hit=='0')&(df_new1.event_list=='12'))
df2.show()

+----------+---------+---------+----------+----------+---------------+--------------+---+-----------+---------------+----------------+-----------+
|visit_high|visit_low|visit_num|event_list|hit_source|post_visit_high|post_visit_low| ds|exclude_hit|visit_hi_id_new|visit_low_id_new|session_cnt|
+----------+---------+---------+----------+----------+---------------+--------------+---+-----------+---------------+----------------+-----------+
+----------+---------+---------+----------+----------+---------------+--------------+---+-----------+---------------+----------------+-----------+



In [114]:
df2=df_new1.where(col("ds").between('01-07-2023','31-07-2023')).select("ds")
df2.show()

+----------+
|        ds|
+----------+
|01-07-2023|
|02-07-2023|
|03-07-2023|
|04-07-2023|
|05-07-2023|
|06-07-2023|
|07-07-2023|
|08-07-2023|
|09-07-2023|
|10-07-2023|
|11-07-2023|
|12-07-2023|
|13-07-2023|
|14-07-2023|
|15-07-2023|
|16-07-2023|
|17-07-2023|
|18-07-2023|
|19-07-2023|
|20-07-2023|
+----------+
only showing top 20 rows



In [90]:
df_new=df.withColumn("visit_hi_id_new",expr("CASE WHEN visit_high='0' or visit_high IS NULL THEN post_visit_high ELSE Visit_high END"))\
         .withColumn("visit_low_id_new",expr("CASE WHEN visit_low='0' or visit_low IS NULL THEN post_visit_low ELSE Visit_low END"))
df_new1=df_new.withColumn("session_cnt", expr("CASE WHEN hit_source NOT IN ('5','7','8','9') then concat(visit_hi_id_new,'-',visit_low_id_new,'-',visit_num) ELSE NULL END")) 
#df22=df_new1.withColumn("ATC_RATE",col("event_list")==11).filter(col("ATC_RATE")==True)
#df22=df_new1.withColumn("ATC_RATE",df_new1.ds.between('01-07-2023','31-07-2023')&(df_new1.exclude_hit=='0')&(df_new1.event_list=='11'))  
#df22=df2.withColumn("ATC_RATE",col("event_list")==12)
#df3=df22.select(countDistinct(col("session_cnt")).alias("distn_cnt"))
#df3.show(truncate=False)


In [115]:
df22=df_new1.filter(col("ds")>="01-07-2023" & col("ds")<="31-07-2023")
#df33=df22.groupby("visit_date").agg(countDistinct(col("session_cnt")))
df22.show()

Py4JError: An error occurred while calling o1076.and. Trace:
py4j.Py4JException: Method and([class java.lang.String]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:1589)

